CREATE WIDGET DROPDOWN ProductionDepartment DEFAULT 'All' CHOICES SELECT productiondepartmentname from reference.productionDepartment
CREATE WIDGET DROPDOWN ResponsibleDepartment DEFAULT 'All' CHOICES SELECT responsibledepartmentname from reference.responsibleDepartment
CREATE WIDGET DROPDOWN Division DEFAULT 'All' CHOICES SELECT division from reference.division
CREATE WIDGET DROPDOWN BusinessUnit DEFAULT 'All' CHOICES SELECT businessunit from reference.businessUnit
CREATE WIDGET DROPDOWN Workflow DEFAULT 'All' CHOICES SELECT workflowname from reference.workflow
CREATE WIDGET DROPDOWN Step DEFAULT 'All' CHOICES SELECT stepname from reference.workflowsteps where workflowname in(:Workflow)
CREATE WIDGET DROPDOWN AgencyState DEFAULT 'All' CHOICES SELECT state from reference.agencyState
CREATE WIDGET DROPDOWN AgencyRegion DEFAULT 'All' CHOICES SELECT regionname from reference.agencyRegion
CREATE WIDGET DROPDOWN Copmany DEFAULT 'All' CHOICES SELECT companyname from reference.company

In [0]:
%sql
with priorDue as (select
       coalesce(productiondepartmentname, 'ERROR') productionDepartmentName,
       coalesce(responsibledepartmentname,'ERROR') responsibleDepartmentName,
       coalesce(businessunitname, 'ERROR')   BusinessUnitName,
       coalesce(divisionname, 'ERROR')       DivisionName,
       coalesce(stepname, 'ERROR')           stepName,
       coalesce(workflowname, 'ERROR')       workflowName,
       coalesce(companyname, 'ERROR')       companyName,
      coalesce(agencyState, 'ERROR')       agencyState,
      coalesce(RegionName, 'ERROR')       agencyRegion,
       Sum(CASE
             WHEN excludedfrommaster = 1 THEN 1
             ELSE 0
           END) as internalPriorDue,
       Sum(CASE
            WHEN excludedfrommaster = 0 THEN 1
             ELSE 0
           END) as externalPriorDue
from gold.workflow
WHERE  CompletedOrPending = 'Pending'
     and (divisionName in(:Division) or :Division='All')
and (businessUnitName in(:BusinessUnit) or :BusinessUnit='All')
and (productiondepartmentname in(:ProductionDepartment) or :ProductionDepartment='All')
and (responsibledepartmentname in(:ResponsibleDepartment) or :ResponsibleDepartment='All')
and (workflowName in(:Workflow) or :Workflow='All')
and (stepName in(:Step) or :Step='All')
and (companyname in(:Company) or :Company='All')
and (dueDateDate between 
      case when :StartDate_MMDDYYYY != 'Now' then :StartDate_MMDDYYYY else current_date() end 
    and 
     case when :EndDate_MMDDYYYY != 'Now' then :EndDate_MMDDYYYY else current_date() end) 
and (agencystate in(:AgencyState) or :AgencyState='All') 
and (regionName in(:AgencyRegion) or :AgencyRegion='All')
GROUP  BY coalesce(productiondepartmentname, 'ERROR') ,
       coalesce(responsibledepartmentname,'ERROR'),
       coalesce(businessunitname, 'ERROR'),
       coalesce(divisionname, 'ERROR'),
       coalesce(stepname, 'ERROR'),
       coalesce(workflowname, 'ERROR'),
       coalesce(companyname, 'ERROR') ,
      coalesce(agencyState, 'ERROR') ,
      coalesce(RegionName, 'ERROR'))
select productiondepartmentName, responsibledepartmentname, businessunitname, divisionname, stepname, workflowname, internalPriorDue, externalpriordue
from priorDue


In [0]:
pandas_df=_sqldf.toPandas()
df=pandas_df.to_json()
dbutils.notebook.exit(df)